# Brain Training
This is the first notebook where everything regarding NF Training will come together

## Preparation I: modules and where to put files
- double check all these parameters
- change sub and sess according to the current participant and session (!)
- double check the destination directory to be sure
- don't use the Template Notebook (if you do so...), but copy/paste a notebook into a subject/session specific directory!
- do not do this work just before a measurement. Have it prepared (and preferable tested!)

In [1]:
# import (most of the) modules that we need
import time
import re
import os
from multiprocessing import Process
from datetime import datetime
import numpy as np
import pylsl
from nftools import guis
import mne
import dynarray
import pickle

In [2]:
# use the qt event loop, disable warnings (they flood the screen)
%matplotlib qt  
# %gui qt

import warnings; warnings.filterwarnings('ignore') 

## Define sub, session and run numbers

In [3]:
# change these according to which sub-sess we have:
sub = 1
sess = 1
run = 1

In [4]:
# define where we put our stuff - prepare for BIDS Format Style
save_dir = '/home/johan/nf/rawdata/BrainTraining/bids'
this_save_dir = os.path.join(save_dir, 'sub-{:02d}'.format(sub), 'sess-{:02d}'.format(sess), 'eeg')

# we should also ... MAKE this savedir! If it exists, we don't do anything.
if not os.path.exists(this_save_dir):
    os.makedirs(this_save_dir)
    print("Directory " , this_save_dir ,  " Created ")
else:    
    print("Directory " , this_save_dir ,  " already exists")   
    
    
# expected files to be read/written:
# 
# fname_raw_eo_run1
# fname_raw_ec_run2
# fname_ica_ocular_rejection

Directory  /home/johan/nf/rawdata/BrainTraining/bids/sub-01/sess-01/eeg  already exists


## Preparion, II: Fix/check the EEG Cap

- fix the EEG Cap
- check with openBCI GUI if the signals look OK, once they do:
- run the `python raw_eo_data --stream`, followed by `/start`
- the light on the usb stick should go <font color="red">RED</font>
- (re)-start the Cap or USB if it doesn't work, followed by commands above

## Connect to the real-time Data Stream

In [5]:
# prints out which streams are currently available
stream_ids = [ pylsl.stream_inlet(s).info().source_id() for s in pylsl.resolve_streams() ]
print(stream_ids)

['openbci_aux_id92', 'openbci_eeg_id92']


- copy/paste the eeg lab name (left of the 2 outputs) into stream_id variable:
- thake the one that says **eeg**, not aux!

In [6]:
# 'subscribe' to a data stream; grab all essential(s), fs, names, etc
# stream_id = 'openbci_eeg_id134'

eeg_stream_id = [stream_id for stream_id in stream_ids if re.match('.*_eeg_.*', stream_id)][0]
print(eeg_stream_id)

openbci_eeg_id92


In [7]:
# try grabbing all the information from that stream:
data_stream=pylsl.resolve_byprop("source_id", eeg_stream_id, timeout=5.0)
if data_stream:
    data_inlet=pylsl.stream_inlet(data_stream[0], max_buflen=10)
    stream_info = data_inlet.info()
    stream_Fs = stream_info.nominal_srate()
    stream_xml = stream_info.desc()
    chans_xml = stream_xml.child("channels")
    chan_xml_list = []
    ch = chans_xml.child("channel")
    while ch.name() == "channel":
        chan_xml_list.append(ch)
        ch = ch.next_sibling("channel")
    channel_names = [ch_xml.child_value("label") for ch_xml in chan_xml_list]
    data_inlet_dt = data_inlet.time_correction(timeout=5.0)
    sampling_freq = data_stream[0].nominal_srate()
    print('name = %s' % data_stream[0].name())
    print('sampling_freq = %d' % sampling_freq)
    print('channel_count = %d' % data_stream[0].channel_count())
    print('channel_format = %d' % data_stream[0].channel_format())
else:
    raise Exception('No Data Stream Found - Is your EEG Cap running?')

name = openbci_eeg
sampling_freq = 125
channel_count = 16
channel_format = 1


## Measure Some Data (Eye Open)
- Inform that the following measurement is an eye open measurement.
- Subjects are allowed to blink as normal
- duration about 2 minutes
- might have to re-run this in order to fix a channel from being busted
- when ready press 'stop acquisition' and close window

In [8]:
# we put the incoming data in here:
np_eo = dynarray.DynamicArray((None, len(channel_names)))

# init/open the window in which we visualize data
w=guis.AcquireData(sampling_freq, channel_names)

# before we start, pull everything from the buffer (empty it)
data_inlet.pull_chunk()
while data_inlet.samples_available(): data_inlet.pull_chunk() 
    
# then start acquiring data as long as button 'stop' not pressed:
w.RUNLOOP=True
while w.RUNLOOP:

    if not data_inlet.samples_available():
        w.update(None)
    else:
        
        chunk_data, chunk_times = data_inlet.pull_chunk(timeout=0.0) # grab from LSL

        np_eo.extend(chunk_data) # add to our list
        w.update(chunk_data) # update the GUI window        

In [9]:
# and we make the MNE data file from it
raw_eo = mne.io.RawArray(np.transpose(np_eo)*1E-6,
                        mne.create_info(channel_names, 
                                    sampling_freq, 
                                    'eeg', 
                                    'standard_1020',
                                    'WARNING'),
                       )

Creating RawArray with float64 data, n_channels=16, n_times=2788
    Range : 0 ... 2787 =      0.000 ...    22.296 secs
Ready.


## Inspect the eo data  (run1)
- scroll through the data see if the EEG signal is what you expect
- mark bad channels (that cannot be rescued)
- mark bad segments

In [10]:
# we use a simple display filter
raw_eo.filter(1, 35, verbose='WARNING').plot();

- save this raw data to disk (filename taken care of)

In [11]:
this_raw_fname = 'sub-{:02d}_sess-{:02d}_task-{}_run-{:02d}.fif'.format(sub, sess, 'eo', run)
fname_raw_eo_run1 = os.path.join(this_save_dir, this_raw_fname)

raw_eo.save(fname_raw_eo_run1, overwrite=True)

Overwriting existing file.
Writing /home/johan/nf/rawdata/BrainTraining/bids/sub-01/sess-01/eeg/sub-01_sess-01_task-eo_run-01.fif
Closing /home/johan/nf/rawdata/BrainTraining/bids/sub-01/sess-01/eeg/sub-01_sess-01_task-eo_run-01.fif [done]


## Measure Some Data (Eye Closed)
- Inform your subject to keep the eye closed, then start the measurement
- duration about 1 minute

In [12]:
# we put the incoming data in here:
np_ec = dynarray.DynamicArray((None, len(channel_names)))

# init/open the window in which we visualize data
w=guis.AcquireData(sampling_freq, channel_names)

# before we start, pull everything from the buffer (empty it)
data_inlet.pull_chunk()
while data_inlet.samples_available(): data_inlet.pull_chunk() 
    
# then start acquiring data as long as button 'stop' not pressed:
w.RUNLOOP=True
while w.RUNLOOP:

    if not data_inlet.samples_available():
        w.update(None)
    else:
        
        chunk_data, chunk_times = data_inlet.pull_chunk(timeout=0.0) # grab from LSL

        np_ec.extend(chunk_data) # add to our list
        w.update(chunk_data) # update the GUI window        

In [13]:
# and we make the MNE data file from it        
raw_ec = mne.io.RawArray(np.transpose(np_ec)*1E-6,
                        mne.create_info(channel_names, 
                                    sampling_freq, 
                                    'eeg', 
                                    'standard_1020',
                                    'WARNING')
                       )
raw_ec.info['bads'] = raw_eo.info['bads']
print('setting bad chanels in the data set to be the same as previous dataset')
print(raw_ec.info['bads'])
print('Do not change bad channels further - re-do eo and ec if needed!')

Creating RawArray with float64 data, n_channels=16, n_times=2247
    Range : 0 ... 2246 =      0.000 ...    17.968 secs
Ready.
setting bad chanels in the data set to be the same as previous dataset
['O1', 'O2', 'C3', 'T5', 'Fp1', 'P4']
Do not change bad channels further - re-do eo and ec if needed!


## Inspect the ec data
- scroll through the data see if the EEG signal is what you expect
- mark bad channels (that cannot be rescued)
- mark bad segments

In [14]:
# we use a simple display filter
raw_ec.filter(1, 35, verbose='WARNING').plot();

- save this raw data to disk (filename taken care of)

In [15]:
this_raw_fname = 'sub-{:02d}_sess-{:02d}_task-{}_run-{:02d}.fif'.format(sub, sess, 'ec', run)
fname_raw_ec_run = os.path.join(this_save_dir, this_raw_fname)

raw_ec.save(fname_raw_ec_run, overwrite=True)

Overwriting existing file.
Writing /home/johan/nf/rawdata/BrainTraining/bids/sub-01/sess-01/eeg/sub-01_sess-01_task-ec_run-01.fif
Closing /home/johan/nf/rawdata/BrainTraining/bids/sub-01/sess-01/eeg/sub-01_sess-01_task-ec_run-01.fif [done]


## Handle the Ocular Artifacts
- this will automatically read in the EO data  (run1)
- and run the ICA analysis
- your job is to select the component most resembling ocular artifact
- and the matrix will/should be saved

In [16]:
# run the ICA calculation; analysis taken from 8_1_SingleRun.ipynb
# extracting the ch names and time information
sampling_freq = raw_eo.info['sfreq']
bad_channels = raw_eo.info['bads']
bad_segments = [(a['onset'], a['duration']) for a in raw_eo.annotations if re.search('BAD', a['description'])]
print(bad_channels)
for s in bad_segments: print('%.2g, %.2g' % (s[0], sum(s)))

# convert to ch mask; and samples to keep/remove
bad_channel_indices = [i for i, ch in enumerate(raw_eo.info['ch_names'])  if ch in bad_channels]
bad_channel_mask = [ch not in bad_channels for ch in raw_eo.ch_names]

# display some information
[range(int(b * sampling_freq), int((b+d) * sampling_freq)) for b, d in bad_segments]

if bad_segments:
    bad_segment_samples = np.concatenate([range(int(b * sampling_freq), int((b+d) * sampling_freq)) for b, d in bad_segments])
else:
    bad_segment_samples=[]

print('channels to remove: %s' % bad_channel_indices)
print('channel mask: ' + repr(bad_channel_mask))
print('number of samples to remove: %s' % len(bad_segment_samples))

# remove the bad channels and bad samples, make new data matrices:
np_eo_forica = raw_eo.copy().get_data().T

np_eo_forica = np.delete(np_eo_forica, bad_segment_samples, axis=0)
np_eo_forica = np.delete(np_eo_forica, bad_channel_indices, axis=1)

print('raw original shape: %d, %d' % raw_eo.get_data().shape)
print('np new shape: %d, %d' % np_eo_forica.shape)

# run the ICA analysis:
from pynfb.protocols.ssd.topomap_selector_ica import ICADialog

ica_rejection, _, _, ica_unmixing_matrix, _, _ = ICADialog.get_rejection(
    np_eo_forica, 
    [n for n in raw_eo.ch_names if n not in bad_channels], 
    sampling_freq,
    decomposition=None
)

ica_rejection = ica_rejection.expand_by_mask(bad_channel_mask)

print('Created an ICA Spatial Filter')
print(ica_rejection)



# loading the ICA:
# with open(fname_ica_ocular_rejection, 'rb') as f: ica_rejection = pickle.load(f)

['O1', 'O2', 'C3', 'T5', 'Fp1', 'P4']
21, 22
channels to remove: [0, 2, 4, 6, 7, 15]
channel mask: [False, True, False, True, False, True, False, False, True, True, True, True, True, True, True, False]
number of samples to remove: 189
raw original shape: 16, 2788
np new shape: 2599, 10
apply filter: 3 to 45
Dropped 29 outliers
Creating RawArray with float64 data, n_channels=10, n_times=2570
    Range : 0 ... 2569 =      0.000 ...    20.552 secs
Ready.
Fitting ICA to data using 10 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 10
Computing Extended Infomax ICA
Fitting ICA took 1.3s.
ICA/CSP time elapsed = 1.370880126953125s
Table drawing time elapsed = 2.414292573928833s
Created an ICA Spatial Filter


## Save the ocular Rejection
- close all open windows

In [17]:
# we save the ICA for ocular rejection:
# save the ICA rejection 
this_raw_fname = 'sub-{:02d}_sess-{:02d}_ica-ocular-rejection_run-{:02d}.pkl'.format(sub, sess, run)
fname_ica_ocular_rejection = os.path.join(this_save_dir, this_raw_fname)

with open(fname_ica_ocular_rejection, 'wb') as f: pickle.dump(ica_rejection, f)
print('saved: ' + fname_ica_ocular_rejection)

# save it also as a .txt matrix (for matlab)
np.savetxt(re.sub('.pkl$','.txt', fname_ica_ocular_rejection), ica_rejection.val)

saved: /home/johan/nf/rawdata/BrainTraining/bids/sub-01/sess-01/eeg/sub-01_sess-01_ica-ocular-rejection_run-01.pkl


## Work On Alpha Power Suppression with CSP

In [18]:
# - load in dataset ec

# check bad channels in eo, compare to ec
if not raw_ec.info['bads'] == raw_eo.info['bads']:
    raise Exception('Bad Channels are not the same between the two datasets - fix this first')

# sort out the bad channels and bad segments over EO and EC data we recorded earlier:
print(bad_channels)
bad_channel_indices = [i for i, ch in enumerate(raw_eo.info['ch_names'])  if ch in bad_channels]
print(bad_channel_indices)
bad_channel_mask = [ch not in bad_channels for ch in raw_eo.ch_names]

bad_segments_eo = [(a['onset'], a['duration']) for a in raw_eo.annotations if re.search('BAD', a['description'])]
for s in bad_segments_eo: print('%.2g, %.2g' % (s[0], sum(s)))
bad_segments_ec = [(a['onset'], a['duration']) for a in raw_ec.annotations if re.search('BAD', a['description'])]
for s in bad_segments_ec: print('%.2g, %.2g' % (s[0], sum(s)))

if bad_segments_eo:
    bad_segment_samples_eo = np.concatenate([range(int(b * sampling_freq), int((b+d) * sampling_freq)) for b, d in bad_segments_eo])
else:
    bad_segment_samples_eo=[]
if bad_segments_ec:
    bad_segment_samples_ec = np.concatenate([range(int(b * sampling_freq), int((b+d) * sampling_freq)) for b, d in bad_segments_ec])
else:
    bad_segment_samples_ec=[]
print('{} bad samples in eyes open'.format(len(bad_segment_samples_eo)))    
print('{} bad samples in eyes closed'.format(len(bad_segment_samples_ec)))   

# copy, remove bad samples, apply ocular rejection, remove bad channels:
np_eo_for_csp = raw_eo.copy().filter(3, 45, verbose='WARNING').get_data().T
np_eo_for_csp = np.delete(np_eo_for_csp, bad_segment_samples, axis=0)
np_eo_for_csp = ica_rejection.apply(np_eo_for_csp)
np_eo_for_csp = np.delete(np_eo_for_csp, bad_channel_indices, axis=1)

np_ec_for_csp = raw_ec.copy().filter(3, 45, verbose='WARNING').get_data().T
np_ec_for_csp = np.delete(np_ec_for_csp, bad_segment_samples, axis=0)
np_ec_for_csp = ica_rejection.apply(np_ec_for_csp)
np_ec_for_csp = np.delete(np_ec_for_csp, bad_channel_indices, axis=1)

# concatenate it for entry into CSP removal (alpha power)
data_for_csp = np.vstack((np_eo_for_csp, np_ec_for_csp))

# make the 'labels' vector: 0 for length of eo; 1 for length of ec.
n_timepoints_in_eo = np_eo_for_csp.shape[0]
n_timepoints_in_ec = np_ec_for_csp.shape[0]
labels_for_csp = np.hstack((np.zeros(n_timepoints_in_eo), np.ones(n_timepoints_in_ec)))

['O1', 'O2', 'C3', 'T5', 'Fp1', 'P4']
[0, 2, 4, 6, 7, 15]
21, 22
17, 18
189 bad samples in eyes open
165 bad samples in eyes closed


In [19]:
# bring up the GUI for CSP filtering of the data:
csp_rejection, filter, topography, _, bandpass, to_all = ICADialog.get_rejection(
    data_for_csp,
    [n for n in raw_eo.ch_names if n not in bad_channels], 
    sampling_freq,
    mode='csp', 
    _stimulus_split=False,
    labels=labels_for_csp, # will convert to 0-1 vector for each sample in x
    marks=None)



apply filter: 3 to 45
Dropped 28 outliers
ICA/CSP time elapsed = 0.05384397506713867s
Table drawing time elapsed = 2.5578622817993164s


## Save the ocular Rejection
- close all open windows

In [20]:
# we save the ICA for ocular rejection:
# save the ICA rejection 
this_raw_fname = 'sub-{:02d}_sess-{:02d}_csp-alpha-rejection_run-{:02d}.pkl'.format(sub, sess, run)
fname_csp_alpha_rejection = os.path.join(this_save_dir, this_raw_fname)

with open(fname_csp_alpha_rejection, 'wb') as f: pickle.dump(csp_rejection, f)
print('saved: ' + fname_csp_alpha_rejection)

# save it also as .txt matrix (for matlab)
np.savetxt(re.sub('.pkl$','.txt', fname_csp_alpha_rejection), csp_rejection.val)

saved: /home/johan/nf/rawdata/BrainTraining/bids/sub-01/sess-01/eeg/sub-01_sess-01_csp-alpha-rejection_run-01.pkl


# Neurofeedback

### Start up the Stimulus on Computer
- connect via WiFi
    - ip address = 10.42.0.1
    - host = stim-pc
    - password = PASSWORD, OR 12345678
- ideally this should already be up and running, so you can skip over these more fast

In [22]:
from callpyff import bcinetwork, bcixml
bcinet = bcinetwork.BciNetwork('10.42.0.1', bcinetwork.FC_PORT, bcinetwork.GUI_PORT, 'bcixml')

In [19]:
print(bcinet.getAvailableFeedbacks())

['TestD2', 'MovingRhomb', 'EOEC', 'LibetClock', 'BrainWaveTraining_II', 'TobiQLAdapter', 'VisualOddballVE', 'EyetrackerFeedback', 'HexoSpeller', 'P300_Rectangle', 'ERPHex', 'BrainWaveTraining', 'StopVigilanceTask', 'FeedbackCursorArrow', 'TrivialPong', 'CheckerboardVEP', 'HexoSpellerVE', 'BoringClock', 'nback_verbal', 'Lesson01', 'BrainPong', 'CakeSpellerVE', 'MovingRhombGL', 'RestingState', 'NFBasicThermometer', 'RSVPSpeller', 'EEGfMRILocalizer', 'MultiVisualOddball', 'Lesson01b', 'GoalKeeper', 'CenterSpellerVE', 'Oddball', 'EyetrackerRawdata', 'StroopFeedback', 'ERPMatrix', 'Lesson04', 'Lesson05', 'Lesson06', 'VisualOddball', 'Lesson02', 'Lesson03']


In [20]:
bcinet.send_init('BrainWaveTraining_II')

In [21]:
# tell the stimulus about the monitor
bcinet.send_signal(bcixml.BciSignal({'EX_TESTNFNOISE': False},None, bcixml.INTERACTION_SIGNAL))
bcinet.send_signal(bcixml.BciSignal({'MONITOR_PIXWIDTH': 1366},None, bcixml.INTERACTION_SIGNAL))
bcinet.send_signal(bcixml.BciSignal({'MONITOR_PIXHEIGHT': 768},None, bcixml.INTERACTION_SIGNAL))
bcinet.send_signal(bcixml.BciSignal({'MONITOR_FULLSCR': True},None, bcixml.INTERACTION_SIGNAL))

In [22]:
from nftools.nftools import signaltracking

In [23]:
# these objects can send variables over to the stimulus
# parameters to convert the filtered EEG signal to the stimulus
# the following parameter should come out of the EEG data, as our initial threshold to use:
global_std_band=5

from nftools.nftools import signaltracking
track_for_eeg_stimuli = signaltracking.sending_to_nfstim(
    sampling_freq,
    thr=1.0 * global_std_band, 
    dur=0.20, 
    feedback_type='eeg', 
    max4audio=1.2, 
    bcinet=bcinet, 
    st_scaling=5 * global_std_band,
    verbose=False
)

# parameters to convert the filtered EMG signal to the stimulus
track_for_emg_stimuli = signaltracking.sending_to_nfstim(
    sampling_freq,
    thr=50 * global_std_band, 
    dur=0.15, 
    feedback_type='emg', 
    bcinet=bcinet, 
    st_scaling=100 * global_std_band,
    verbose=False
)

thr: 5.00, dur: 0.20
bcinet is passed on
thr: 250.00, dur: 0.15
bcinet is passed on


### Start the process that will monitor events from the Stimulation Laptop
 - this will listen on UDP port 6500 for any incoming markers
 - This is to convert signals from the Presentation into annotations
 - grab markers in the NF loop with: `while not marker_queue.empty():`

In [34]:
# tell the stimulus computer where markers need to be sent to:
import socket
ip_address = socket.gethostbyname(socket.getfqdn())
port = 6500
print(ip_address+":"+str(port))
bcinet.send_signal(bcixml.BciSignal({'EVENT_destip': ip_address},None, bcixml.INTERACTION_SIGNAL))
bcinet.send_signal(bcixml.BciSignal({'EVENT_destport': port},None, bcixml.INTERACTION_SIGNAL))

10.3.97.246:6500


In [14]:
from multiprocessing import Process, Queue, Event
import time
if not 'marker_queue' in locals():
    marker_queue = Queue()
if not 'stop_server' in locals():
    stop_server = Event()
from libmushu.ampdecorator import marker_reader

stop_server.set()
time.sleep(0.5)
stop_server.clear()

tcp_reader = Process(target=marker_reader, args=(
    marker_queue,
    Event().set(),
    Event(),
    stop_server,
    6500
    )
)

tcp_reader.start()
# stop the server with: 
# stop_server.set()
# tcp_reader.join()

starting markerserver on port: 6500


### Start the NF Stimulation
 - you still have to press <ENTER> to actually start the stimulus, but Not Yet!!

In [4]:
bcinet.play()

NameError: name 'bcinet' is not defined

# Definition of the Real-Time analysis Steps

In [36]:
from pynfb.signal_processing.filters import (FilterSequence, 
                                             CFIRBandEnvelopeDetector, 
                                             ExponentialSmoother,
                                             SpatialFilter,
                                             ButterFilter,
                                             ButterBandEnvelopeDetector,
                                             ScalarButterFilter,
                                             MASmoother,
                                             FFTBandEnvelopeDetector,
                                            )

In [14]:
# Define Filter Sequence for NF

# Which channel do we select for the EEG - we do C3.
rt_eeg_channels = ['C3']
rt_eeg_channels_mask = np.where([ch in rt_eeg_channels for ch in channel_names], 1, 0)/len(rt_eeg_channels)

preprocess_filters_eeg = FilterSequence([
    ica_rejection,
    csp_rejection,
    SpatialFilter(rt_eeg_channels_mask),
])
envelope_filter_eeg = CFIRBandEnvelopeDetector([12, 15], sampling_freq, MASmoother(150))
butter_visualization_eeg = ButterFilter([12, 15], sampling_freq, 1)


# Processing of the EMG - this is basically our second channel...
rt_emg_channels = ['T3','T4','Fp1','Fp2']
rt_emg_channels_mask = np.where([ch in rt_emg_channels for ch in channel_names], 1, 0)/len(rt_emg_channels)
preprocess_filters_emg = FilterSequence([
    SpatialFilter(rt_eeg_channels_mask),
])
envelope_filter_emg = ButterBandEnvelopeDetector([55, 56], sampling_freq, MASmoother(150))
butter_visualization_emg = ButterFilter([55, 65], sampling_freq, 1)

In [15]:
# create our UI 'Experience' -- it can consist of 3 separate, movable windows (for now)
# same window as before + 2 other windows - 1 for interaction with stim/thresholds; 1 for looking
# at the analysis itself.
w_acquire = guis.AcquireData(sampling_freq, channel_names)
w_interaction = guis.NFChangeThresholds(track_for_eeg_stimuli, track_for_emg_stimuli)
w_eeganalysis = guis.AnalyzeData(sampling_freq, ['EEG','env','thr','vmarker','amarker'],track_for_eeg_stimuli)
w_emganalysis = guis.AnalyzeData(sampling_freq, ['EMG','env','thr','vmarker','amarker'],track_for_eeg_stimuli)

# Press start on the NF Stimulation Laptop now

In [16]:
# clear EEG Data buffer
data_inlet.pull_chunk()
while data_inlet.samples_available(): data_inlet.pull_chunk() 

In [17]:
# containers for data collection: 
time_nf = dynarray.DynamicArray()
data_nf = dynarray.DynamicArray((None, len(channel_names)))
data_analysis_eeg = dynarray.DynamicArray((None, 5))
data_analysis_emg = dynarray.DynamicArray((None, 5))

# collect markers from the Stimulation:
stim_Annotations = mne.Annotations(0, 0, 'Start NF Loop')

# collect markers from the interaction GUI:
w_interaction.GUI_Annotations.append(time.time()-w_interaction.begin_time, 0, 'startloop')

# start the loop
w_acquire.RUNLOOP=True
acquisition_start = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
while w_acquire.RUNLOOP:
    
    # check if the presentation gave any markers, and collect them
    while not marker_queue.empty():
        stim_time, stim_code = marker_queue.get()
        stim_Annotations.append(stim_time, 0, stim_code)
    
    
    if not data_inlet.samples_available(): 
        w_acquire.update(None)
    else:
        chunk_data, chunk_times = data_inlet.pull_chunk() # grab from LSL

        # update signal window so we can see raw signals
        w_acquire.update(chunk_data) 
        
        # store the raw data (and times)
        time_nf.extend(chunk_times)
        data_nf.extend(chunk_data)

        # store the markers of the Stimulus computer
        while not marker_queue.empty():
            stim_time, stim_code = marker_queue.get()
            stim_Annotations.append(stim_time, 0, stim_code)

        
        #
        # EEG Signal processing
        #
        
        # apply spatial and temporal filters to the raw signal; for EEG and EMG:
        preprocessed_eeg = preprocess_filters_eeg.apply(chunk_data)        
        envelope_eeg = envelope_filter_eeg.apply(preprocessed_eeg)


        # check if it's above threshold or not; send markers to stimulus computer
        visual_markers_eeg, audio_markers_eeg = track_for_eeg_stimuli.check_above_threshold(envelope_eeg)  
        # send the signal  to stimulus computer, too
        track_for_eeg_stimuli.send_data_signal(envelope_eeg) 
        

        # visualize the processing steps
        analysis_names_eeg = ('EEG', 'env', 'thr', 'vmarker', 'amarker')
        analysis_data_eeg = np.vstack((
            np.abs(butter_visualization_eeg.apply(preprocessed_eeg[:, None])[:, 0]),
            envelope_eeg,
            track_for_eeg_stimuli.thr * np.ones(preprocessed_eeg.shape),
            visual_markers_eeg,
            audio_markers_eeg,
        )).T
        
        data_analysis_eeg.extend(analysis_data_eeg)  # store it for later conversion
        w_eeganalysis.update(analysis_data) # update analysis window

        
        #
        # EMG Signal Processing
        #
        
        # do the same for the EMG NF, too:
        # apply spatial and temporal filters to the raw signal; for EEG and EMG:
        preprocessed_emg = preprocess_filters_emg.apply(chunk_data)
        envelope_emg = envelope_filter_emg.apply(preprocessed_emg)
        
        # check if it's above threshold or not; send markers to stimulus computer
        visual_markers_emg, audio_markers_emg = track_for_emg_stimuli.check_above_threshold(envelope_emg) 
        # send the signal  to stimulus computer, too
        track_for_emg_stimuli.send_data_signal(envelope_emg) 

        # visualize the processing steps
        analysis_names_eeg = ('EMG', 'env', 'thr', 'vmarker', 'amarker')
        analysis_data_emg = np.vstack((
            np.abs(butter_visualization_emg.apply(preprocessed_emg[:, None])[:, 0]),
            envelope_emg,
            track_for_eeg_stimuli.thr * np.ones(preprocessed_emg.shape),
            visual_markers_emg,
            audio_markers_emg,
        )).T
        
        data_analysis_emg.extend(analysis_data)  # store it for later conversion
        w_emganalysis.update(analysis_data) # update analysis window


## save all the gathered data to disk

In [13]:
# and we make the MNE data file from it        
raw_nftraining = mne.io.RawArray(np.transpose(np_ec)*1E-6,
                        mne.create_info(channel_names, 
                                    sampling_freq, 
                                    'eeg', 
                                    'standard_1020',
                                    'WARNING')
                       )
# saving the raw data
this_raw_fname = 'sub-{:02d}_sess-{:02d}_task-{}_run-{:02d}.fif'.format(sub, sess, 'raw-nftraining', run)
fname_raw_nftraining_run = os.path.join(this_save_dir, this_raw_fname)

raw_nftraining.save(fname_raw_ec_run, overwrite=True)

Creating RawArray with float64 data, n_channels=16, n_times=2247
    Range : 0 ... 2246 =      0.000 ...    17.968 secs
Ready.
setting bad chanels in the data set to be the same as previous dataset
['O1', 'O2', 'C3', 'T5', 'Fp1', 'P4']
Do not change bad channels further - re-do eo and ec if needed!


In [18]:
# and we make the MNE data file from it
rtanalyzed_nftraining_eeg = mne.io.RawArray(np.transpose(np.array(analysis_names_eeg) * [1E-6, 1E-6, 1E-6, 1, 1]),
                        mne.create_info(analyzed_nftraining_eeg, 
                                    sampling_freq, 
                                    ['eeg','eeg','eeg','stim','stim'], 
                                    None)
                       )
this_raw_fname = 'sub-{:02d}_sess-{:02d}_task-{}_run-{:02d}.fif'.format(sub, sess, 'rtanalyzed-nftraining-eeg', run)
fname_rtanalyzed_nftraining_eeg = os.path.join(this_save_dir, this_raw_fname)


# handle annotations, since w_interaction starts before data acquisition
annots = w_interaction.GUI_Annotations.copy()
tdelta = datetime.strptime(acquisition_start,'%Y-%m-%d %H:%M:%S.%f').timestamp() - annots.orig_time
annots.onset -= tdelta
annots.orig_time=None

starting_annots = annots[list(map(lambda d: re.match('start.*', d) is not None, annots.description))]
starting_annots.onset = 0.1*np.ones(len(starting_annots))
starting_annots.orig_time = None

# handle also these:
# stim_Annotations ... process them


rtanalyzed_nftraining_eeg.set_annotations(starting_annots + annots)
rtanalyzed_nftraining_eeg.save(fname_rtanalyzed_nftraining_eeg)

Creating RawArray with float64 data, n_channels=5, n_times=10108
    Range : 0 ... 10107 =      0.000 ...    20.214 secs
Ready.


In [ ]:
# and we make the MNE data file from it
analyzed_nftraining_emg = mne.io.RawArray(np.transpose(np.array(analysis_names_emg) * [1E-6, 1E-6, 1E-6, 1, 1]),
                        mne.create_info(analysis_names_emg, 
                                    sampling_freq, 
                                    ['emg','emg','emg','stim','stim'], 
                                    None)
                       )
this_raw_fname = 'sub-{:02d}_sess-{:02d}_task-{}_run-{:02d}.fif'.format(sub, sess, 'nftraining-emg', run)
fname_analyzed_nftraining_emg = os.path.join(this_save_dir, this_raw_fname)

analyzed_nftraining_eeg.save(fname_analyzed_nftraining_emg)